In [1]:
import hail as hl

In [2]:
hl.init()

2022-11-30 18:13:56 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/hail-20221130-1813-0.2.99-57537fea08d4.log


In [3]:
maps_table = hl.read_table(
'gs://janucik-dataproc-stage/01_maps/1-array-rerun_maps_per_variant-separate-models-main_2022_Nov_24_v3_final/maps_downsampling_multi_model.ht'
)


In [4]:
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')
meta_fields = ht.freq_meta.collect()
maps_table = maps_table.annotate_globals(freq_metas = meta_fields)

In [5]:
maps_zipped = maps_table.transmute(
data = hl.zip(maps_table.N_singletons, maps_table.expected_singletons, maps_table.N_variants, maps_table.ps_agg, maps_table.maps, maps_table.maps_sem, maps_table.freq_metas[0])
)

In [6]:
maps_expl = maps_zipped.explode('data')

In [7]:
maps_clean = maps_expl.annotate(
Singletons = maps_expl.data[0],
Expected_Singletons = maps_expl.data[1],
N_variants = maps_expl.data[2],
PS = maps_expl.data[3],
MAPS = maps_expl.data[4],
MAPS_Sem = maps_expl.data[5],
Metadata = maps_expl.data[6].values()
)

In [9]:
maps_clean.show(3)

,,,,,,,,
lof_csq_collapsed,data,Singletons,Expected_Singletons,N_variants,PS,MAPS,MAPS_Sem,Metadata
str,"tuple(int64, float64, int64, float64, float64, float64, dict<str, str>)",int64,float64,int64,float64,float64,float64,array<str>
"""HC""","(98660,7.45e+04,152811,6.46e-01,1.58e-01,1.22e-03,{""group"":""adj""})",98660,7.45e+04,152811,6.46e-01,1.58e-01,1.22e-03,"[""adj""]"
"""HC""","(97002,7.33e+04,152811,6.35e-01,1.55e-01,1.23e-03,{""group"":""raw""})",97002,7.33e+04,152811,6.35e-01,1.55e-01,1.23e-03,"[""raw""]"
"""HC""","(54519,4.30e+04,78756,6.92e-01,1.46e-01,1.64e-03,{""group"":""adj"",""pop"":""nfe""})",54519,4.30e+04,78756,6.92e-01,1.46e-01,1.64e-03,"[""adj"",""nfe""]"


In [8]:
maps_clean_population_ds = maps_clean.annotate(is_raw_adj = ((maps_clean.Metadata[0].contains('raw')) | maps_clean.Metadata[0].contains('adj')))
                                               
maps_clean_population_ds = maps_clean_population_ds.filter(maps_clean_population_ds.is_raw_adj == 0)

maps_clean_population_ds = maps_clean_population_ds.annotate(population = maps_clean_population_ds.Metadata[2],
                                                            downsampling = maps_clean_population_ds.Metadata[0])

maps_clean_population_ds = maps_clean_population_ds.select('Singletons', 'Expected_Singletons', 'N_variants', 'PS', 'MAPS', 'MAPS_Sem', 'downsampling', 'population')

maps_clean_population_ds.export('/Users/adrian/BroadIS/MAPS/01_data/maps_downsampling_multi_model.csv', delimiter = ",")

maps_clean_population_ds.show(3)


2022-11-30 18:15:03 Hail: INFO: merging 5 files totalling 48.8K...  (0 + 4) / 4]
2022-11-30 18:15:03 Hail: INFO: while writing:
    /Users/adrian/BroadIS/MAPS/01_data/maps_downsampling_multi_model.csv
  merge time: 31.250ms


,,,,,,,,
lof_csq_collapsed,Singletons,Expected_Singletons,N_variants,PS,MAPS,MAPS_Sem,downsampling,population
str,int64,float64,int64,float64,float64,float64,str,str
"""HC""",199,1.13e+02,307,6.48e-01,2.79e-01,2.73e-02,"""10""","""global"""
"""HC""",301,1.83e+02,487,6.18e-01,2.43e-01,2.20e-02,"""20""","""global"""
"""HC""",547,3.10e+02,846,6.47e-01,2.80e-01,1.64e-02,"""50""","""global"""
"""HC""",847,4.92e+02,1284,6.60e-01,2.77e-01,1.32e-02,"""100""","""global"""
"""HC""",1225,7.03e+02,1791,6.84e-01,2.91e-01,1.10e-02,"""158""","""global"""
"""HC""",1462,8.33e+02,2122,6.89e-01,2.96e-01,1.00e-02,"""200""","""global"""
"""HC""",2629,1.58e+03,3713,7.08e-01,2.83e-01,7.46e-03,"""456""","""global"""
"""HC""",2848,1.72e+03,3999,7.12e-01,2.82e-01,7.16e-03,"""500""","""global"""


In [23]:
maps_clean_global =  maps_clean.filter(
    (maps_clean.Metadata[0] == 'adj') & (maps_clean.Metadata.length() == 1))

maps_clean_global = maps_clean_global.annotate(
    population = maps_clean_global.Metadata[0])

maps_clean_global = maps_clean_global.select('Singletons', 'Expected_Singletons', 'N_variants', 'PS', 'MAPS', 'MAPS_Sem', 'population')

maps_clean_global.export('/Users/adrian/BroadIS/MAPS/01_data/maps_clean_global.csv', delimiter = ",")

maps_clean_global.show()


2022-11-30 18:36:19 Hail: INFO: merging 5 files totalling 378...
2022-11-30 18:36:19 Hail: INFO: while writing:
    /Users/adrian/BroadIS/MAPS/01_data/maps_clean_global.csv
  merge time: 21.607ms


,,,,,,,
lof_csq_collapsed,Singletons,Expected_Singletons,N_variants,PS,MAPS,MAPS_Sem,population
str,int64,float64,int64,float64,float64,float64,str
"""HC""",98660,7.45e+04,152811,6.46e-01,1.58e-01,1.22e-03,"""adj"""
"""LC""",11631,1.03e+04,20382,5.71e-01,6.75e-02,3.47e-03,"""adj"""
"""missense_variant""",2090809,1.88e+06,3913394,5.34e-01,5.48e-02,2.52e-04,"""adj"""
"""synonymous_variant""",931260,9.31e+05,1942562,4.79e-01,1.80e-16,3.58e-04,"""adj"""
